In [4]:
#https://towardsdatascience.com/reading-an-image-in-python-without-using-special-libraries-534b57de7dd4
import numpy as np
import os.path

def PNG_Open(image_name, width, height):
    fin = open(image_name, 'r')
    image = np.fromfile(fin, dtype = np.uint8)
    
    print("Dimension of the old image array: ", image.ndim)
    print("Size of the old image array: ", image.size) 
    
    print(image)
    
    


image1_name = "primeira-png.png"
image2_name = "segunda-png.png"


width = 100
height = 80
PNG_Open(image1_name, 100, 80)
PNG_Open(image2_name, 100, 80)

Dimension of the old image array:  1
Size of the old image array:  373
[137  80  78  71  13  10  26  10   0   0   0  13  73  72  68  82   0   0
   0 100   0   0   0  80   8   2   0   0   0  96 107  71 184   0   0   0
   1 115  82  71  66   0 174 206  28 233   0   0   0   4 103  65  77  65
   0   0 177 143  11 252  97   5   0   0   0   9 112  72  89 115   0   0
  14 195   0   0  14 195   1 199 111 168 100   0   0   1  10  73  68  65
  84 120  94 237 220 177 141 194  48   0  64 209 112  91  64 193  88  71
 193 124  20 176  70  54 161 224 214 184  20  30  32  95  74  20 138 247
  26 219 157 245 149 184 136 228 156 230 121 158  88 231 103 140 172  32
  86  32  86  32  86  32  86  32  86  32  86  32  86  32  86  32  86  32
  86  32  86  32  86  32  86  32  86  32  86  32  86  32  86 176 239 103
 229 207 251 111 204  14 114 185 158 199 108  11  59 198  90  74 221 238
 191  99 113 144 231 227 181  97 175 189  94 195 111  40 181  88 246 176
 225 211 237 204  10 196  10 196  10 196  10 196  10 

In [11]:
import cv2
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def Image_Open(image_name):
    return cv2.imread(image_name)

def Is_Grey_Scale(image):
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            r, g, b = image[y,x]
            if r != g != b: 
                #print(x, " ", y)
                return False
    return True


def main(image_name, x, y):
    image = Image_Open(image_name)    
        
    print("Altura: ", image.shape[0])
    print("Largura: ", image.shape[1])
    
    if(y < image.shape[0] and x < image.shape[1]):
        if(Is_Grey_Scale(image)):
            print("Canais: ", "1")
            print("")
            print("O pixel na posicao (", x, ", ", y, ") possui valor: ", image[y,x])
        else:
            b, g, r = cv2.split(image)
            print("Canais: ", image.shape[2])
            print("")
            print("O pixel vermelho na posicao (", x, ", ", y, ") possui valor: ", r[y,x])
            print("O pixel verde na posicao (", x, ", ", y, ") possui valor: ", g[y,x])
            print("O pixel azul na posicao (", x, ", ", y, ") possui valor: ", b[y,x])
    
    

interact(main,
         image_name = [('primeira-png','primeira-png.png'),('segunda-png',"segunda-png.png")],
         x = widgets.BoundedIntText(description = 'X Value', value = 31, min = 0),
         y = widgets.BoundedIntText(description = 'Y Value', value =  19, min = 0))

interactive(children=(Dropdown(description='image_name', options=(('primeira-png', 'primeira-png.png'), ('segu…

<function __main__.main(image_name, x, y)>

In [21]:
#https://www.w3.org/TR/PNG/#8Interlace

#https://medium.com/achiev/understanding-and-decoding-png-format-example-in-ts-b31fdde1151b

import zlib
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def PutInAByte(byte):
    if(byte >= 256):
        byte -= 256
    return byte


def PNG_Open(image_name):
    with open(image_name, 'rb') as i:
        image_b = i.read(-1)
    
    pngLenght = len(image_b)
    pngSignature = image_b[:8]
    
    
    #p = int.from_bytes(pngSignature[1:2], "big")
    #print (image_b)
    print("tamanho: ", pngLenght)
    print ("Assinatura: ",pngSignature)
    
    
    dataFromChunks = []
    crcLenght = 4
    index = 8
    #while(index < pngLenght):
    
    #Getting the first chunk
    chunkLength = int.from_bytes(image_b[index: index + 4], "big") 
    index += 4
    chunkType = image_b[index: index + 4].decode("utf-8")
    index += 4
    chunkData = image_b[index : index + chunkLength]
    index += chunkLength
    index += crcLenght
    
    width = int.from_bytes(chunkData[0:4], "big")  
    height = int.from_bytes(chunkData[4:8], "big")  
    bitDepth = chunkData[8]
    colourType = chunkData[9] 
    compressionMethod = chunkData[10]
    filterMethod = chunkData[11] 
    interlaceMethode = chunkData[12] 
    
    print("Largura: ", width)
    print("Altura: ", height)
    
    #Collecting data
    while(index <= pngLenght):
        chunkLength = int.from_bytes(image_b[index: index + 4], "big") 
        index += 4
        chunkType = image_b[index: index + 4].decode("utf-8")
        index += 4
        chunkData = image_b[index : index + chunkLength]
        index += chunkLength
        index += crcLenght
        if(chunkType == "IDAT"):
            dataFromChunks.append(chunkData)
    
    #Compressed data
    #print("dataFromChunks: ", dataFromChunks)
    print("dataFromChunks lenght: ", len(dataFromChunks[0]))
    
    #decompressed data
    decompressPixels = zlib.decompress(dataFromChunks[0])
    print("Tamanho da Imagem: ", len(decompressPixels))
    #print("Imagem: ", decompressPixels)
    
    #creating images channels
    r = np.zeros((height, width)) #channel 0
    g = np.zeros((height, width)) #channel 1
    b = np.zeros((height, width)) #channel 2
    
    #Filling the channels 
    index = 0
    for y in range (0, height):
        #Getting the filter
        filterType = decompressPixels[index]
        index +=1     
        
        for x in range (0, width):
            if(filterType == 1):
                if(x == 0): 
                    r[y, x] = decompressPixels[index]
                    g[y, x] = decompressPixels[index + 1]
                    b[y, x] = decompressPixels[index + 2]
                else:
                    r[y, x] = PutInAByte(decompressPixels[index] + r[y, x - 1])
                    g[y, x] = PutInAByte(decompressPixels[index + 1] + g[y, x - 1])
                    b[y, x] = PutInAByte(decompressPixels[index + 2] + b[y, x - 1])   
                    
            elif(filterType == 2):
                r[y, x] = PutInAByte(decompressPixels[index] + r[y- 1, x])
                g[y, x] = PutInAByte(decompressPixels[index + 1] + g[y- 1, x])
                b[y, x] = PutInAByte(decompressPixels[index + 2] + b[y- 1, x])
                
            elif(filterType == 0):
                r[y, x] = PutInAByte(decompressPixels[index])
                g[y, x] = PutInAByte(decompressPixels[index + 1])
                b[y, x] = PutInAByte(decompressPixels[index + 2])
            else:
                print("Filter type: ", filterType)
            index += 3
    
    print(index)
    #print(r)
    #print(g)
    #print(b)
        
    #merging channels
    output =[]
    output.append(r)
    output.append(g)
    output.append(b)
    return output


#image1 = PNG_Open(image1_name)
#image2 = PNG_Open(image2_name)

def main(image_name, x, y):
    image = PNG_Open(image_name)    
        
    print("Altura: ", image[0].shape[0])
    print("Largura: ", image[0].shape[1])
    
    r = image[0]
    g = image[1]
    b = image[2]
    
    print("O pixel vermelho na posicao (", x, ", ", y, ") possui valor: ", r[y,x])
    print("O pixel verde na posicao (", x, ", ", y, ") possui valor: ", g[y,x])
    print("O pixel azul na posicao (", x, ", ", y, ") possui valor: ", b[y,x])
    
    

interact(main,
         image_name = [('primeira-png','primeira-png.png'),('segunda-png',"segunda-png.png")],
         x = widgets.BoundedIntText(description = 'X Value', value = 31, min = 0),
         y = widgets.BoundedIntText(description = 'Y Value', value =  19, min = 0))

interactive(children=(Dropdown(description='image_name', options=(('primeira-png', 'primeira-png.png'), ('segu…

<function __main__.main(image_name, x, y)>

In [33]:
#https://www.w3.org/TR/PNG/#8Interlace

#Libs to read file
import zlib
import numpy as np

#Libs Just to Display
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def PutInAByte(byte):
    if(byte >= 256):
        byte -= 256
    return byte


def PNG_Open(image_name):
    #reading file's bytes
    with open(image_name, 'rb') as i:
        image_b = i.read(-1)
    
    #Getting file's info
    pngLenght = len(image_b)
    pngSignature = image_b[:8]    
    
    #Getting the first chunk
    dataFromChunks = []
    crcLenght = 4
    index = 8
    
    chunkLength = int.from_bytes(image_b[index: index + 4], "big") 
    index += 4
    chunkType = image_b[index: index + 4].decode("utf-8")
    index += 4
    chunkData = image_b[index : index + chunkLength]
    index += chunkLength
    index += crcLenght
    
    #Getting data from first chunk
    width = int.from_bytes(chunkData[0:4], "big")  
    height = int.from_bytes(chunkData[4:8], "big")  
    bitDepth = chunkData[8]
    colourType = chunkData[9] 
    compressionMethod = chunkData[10]
    filterMethod = chunkData[11] 
    interlaceMethode = chunkData[12] 
    #print("Largura: ", width)
    #print("Altura: ", height)
    
    #Collecting compressed data
    while(index <= pngLenght):
        chunkLength = int.from_bytes(image_b[index: index + 4], "big") 
        index += 4
        chunkType = image_b[index: index + 4].decode("utf-8")
        index += 4
        chunkData = image_b[index : index + chunkLength]
        index += chunkLength
        index += crcLenght
        if(chunkType == "IDAT"):
            dataFromChunks.append(chunkData)
    
    #print("dataFromChunks: ", dataFromChunks)
    #print("dataFromChunks lenght: ", len(dataFromChunks[0]))
    
    #decompressing data
    decompressPixels = zlib.decompress(dataFromChunks[0])
    #print("Tamanho da Imagem: ", len(decompressPixels))
    #print("Imagem: ", decompressPixels)
    
    #creating images channels
    out = np.zeros((height, width, 3)) # 3 channel R, G and B
    
    #Filling the channels 
    index = 0
    for y in range (0, height):
        #Getting the filter
        filterType = decompressPixels[index]
        index +=1     
        
        for x in range (0, width):
            if(filterType == 1):
                if(x == 0): 
                    out[y, x, 0] = decompressPixels[index]
                    out[y, x, 1] = decompressPixels[index + 1]
                    out[y, x, 2] = decompressPixels[index + 2]
                else:
                    out[y, x, 0] = PutInAByte(decompressPixels[index] + out[y, x - 1, 0])
                    out[y, x, 1] = PutInAByte(decompressPixels[index + 1] + out[y, x - 1, 1])
                    out[y, x, 2] = PutInAByte(decompressPixels[index + 2] + out[y, x - 1, 2])   
                    
            elif(filterType == 2):
                out[y, x, 0] = PutInAByte(decompressPixels[index] + out[y- 1, x, 0])
                out[y, x, 1] = PutInAByte(decompressPixels[index + 1] + out[y- 1, x, 1])
                out[y, x, 2] = PutInAByte(decompressPixels[index + 2] + out[y- 1, x, 2])
                
            elif(filterType == 0):
                out[y, x, 0] = PutInAByte(decompressPixels[index])
                out[y, x, 1] = PutInAByte(decompressPixels[index + 1])
                out[y, x, 2] = PutInAByte(decompressPixels[index + 2])
            else:
                print("Filter type: ", filterType)
            index += 3
    return out

def main(image_name, x, y):
    image = PNG_Open(image_name)
    
    print("O pixel vermelho na posicao (", x, ", ", y, ") possui valor: ", image[y, x, 0].astype(int))
    print("O pixel verde na posicao (", x, ", ", y, ") possui valor: ", image[y, x, 1].astype(int))
    print("O pixel azul na posicao (", x, ", ", y, ") possui valor: ", image[y, x, 2].astype(int))
    
    if(image[y, x, 0] + image[y, x, 1] + image[y, x, 2] > (256 * 1.5)):
        image[y, x, 0] = 0
        image[y, x, 1] = 0
        image[y, x, 2] = 0
    else:
        image[y, x, 0] = 255
        image[y, x, 1] = 255
        image[y, x, 2] = 255
    
    plt.imshow(image.astype(int))
    plt.plot()

interact(main,
         image_name = [('primeira-png','primeira-png.png'),('segunda-png',"segunda-png.png")],
         x = widgets.IntSlider(description = 'X Value', value = 31, min = 0, max = 99),
         y = widgets.IntSlider(description = 'Y Value', value =  19, min = 0, max = 79))

interactive(children=(Dropdown(description='image_name', options=(('primeira-png', 'primeira-png.png'), ('segu…

<function __main__.main(image_name, x, y)>